In [4]:
library(tidyverse)
library(tidymodels)
library(repr)
library(dplyr)

In [11]:
heart_data <- read_csv("heart_data.csv") |>
select(chol, trestbps, fbs, age, sex) |>
mutate(sex = as_factor(sex))|>
mutate(sex = fct_recode(sex, "Male" = 1, "Female" = 0))
heart_data

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (14): age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpea...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ERROR: [1m[33mError[39m in `mutate()`:[22m
[1m[22m[36mℹ[39m In argument: `sex = fct_recode(sex, Male = 1, Female = 0)`.
[1mCaused by error in `fct_recode()`:[22m
[1m[22m[33m![39m Each element of `...` must be a named string.
[36mℹ[39m Problems with 2 arguments: Male and Female
